In [33]:
import pandas as pd
import json
import os
import pymongo
import pymatgen
import re

{'PMG_VASP_PSP_DIR': '~/local/pmg_psp/'}


In [5]:
client = pymongo.MongoClient()
entries = client.PyChemiaDB_OQMD12.pychemia_entries


In [6]:
ls = os.listdir('workfunctions/')

In [44]:
df = pd.DataFrame()
df['formula'] = ''
df['SPG'] = ''
df['International'] = ''
df['Termination'] = ''
df['direction'] = ''
df['Mismatch_A %'] = ''
df['Mismatch_B %'] = ''
df['Mismatch_Angle'] = ''
df['Workfunction[eV]'] = ''
df['Efermi[eV]'] = ''
df['icsd'] = ''
df['Link'] = ''
df['_id'] = ''
df['status'] = ''

for item in ls :
    rf = open('results/'+item,'r')
    data_mismatch = json.load(rf)
    rf.close()
    key_mismatch = [x for x in data_mismatch][0]
    if len(data_mismatch[key_mismatch]["[0, 0, 1]"]) != 0:
        direction = "[0, 0, 1]"
    elif len(data_mismatch[key_mismatch]["[0, 1, 0]"]) != 0:
        direction = "[0, 1, 0]"
    elif len(data_mismatch[key_mismatch]["[1, 0, 0]"]) != 0:
        direction = "[1, 0, 0]"
    Mismatch_A = data_mismatch[key_mismatch][direction]['mismatch'][0]
    Mismatch_B = data_mismatch[key_mismatch][direction]['mismatch'][1]
    Mismatch_Angle = data_mismatch[key_mismatch][direction]['mismatch'][2]
    _id = data_mismatch[key_mismatch]['_id']
    oqmd_id = re.findall('[0-9]*_[a-zA-z0-9]*_0*([1-9][0-9]*)',_id)[0]


    entry = entries.find_one({'_id':_id})
    path = entry['properties']['oqmd']['path']
    spg = entry['properties']['oqmd']['sym_dtset']['number']
    international = entry['properties']['oqmd']['sym_dtset']['international']
    st_dict = entry['structure']
    substrate = pymatgen.core.Structure(lattice=st_dict["cell"],
                                             species=st_dict['symbols'],coords=st_dict['positions'],coords_are_cartesian=True)
        
    formula = substrate.composition.reduced_formula
    if path != None and 'icsd' in path :
        icsd = True
    else :
        icsd = False
    rf = open('workfunctions/'+item,'r')
    data_wf = json.load(rf)
    rf.close()
    key_wf = [x for x in data_wf][0]
    for layer in data_wf[key_wf]:
        if data_wf[key_wf][layer] == "interrupted":
            continue
        efermi = data_wf[key_wf][layer]['efermi']
        status = data_wf[key_wf][layer]['status']
        workfunction = data_wf[key_wf][layer]['work-function']
        to_append = {'formula'     : formula,
                     'SPG'         : spg,
                     'International': international,
                     'Termination' : layer,
                     'direction'   : direction,
                     "Mismatch_A %": round(Mismatch_A*100,2),
                     "Mismatch_B %": round(Mismatch_B*100,2),
                     "Mismatch_Angle": Mismatch_Angle,
                     "Workfunction[eV]": workfunction,
                     "Efermi[eV]": efermi,
                     "icsd": icsd,
                     "Link": 'http://oqmd.org/materials/entry/'+ oqmd_id,
                     "_id": _id,
                     "status": status}
        df = df.append(to_append,ignore_index=True)

In [49]:
wf = df['Workfunction[eV]']

In [46]:
df.to_excel('final_results.xlsx')

In [56]:
df['Workfunction[eV]'].describe()



count     4600
unique    4592
top       None
freq         5
Name: Workfunction[eV], dtype: object

In [65]:
wf.describe(include=float)

count     4600
unique    4592
top       None
freq         5
Name: Workfunction[eV], dtype: object